## format and split data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
train = pd.read_csv('train_small.csv')

/home/isabella_uthini_co_za/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.dropna(subset=['type'], inplace=True)

In [4]:
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
train.head()

,channel_type,channel_name,id,date,from,text,from_2,type,category
0,personal_chat,TZ155 Nontobeko Mthembu,226915,2018-03-22T17:59:29,Setup (Master),Sawubona Jessie 😊. ;;We can see that you are e...,setup,noncontent,setup
1,private_group,19:00 (10/07) Thabiso (2)🏆3️⃣,376217,2018-04-03T19:01:47,ThishaBot,Any questions?,bot,noncontent,conv
2,private_group,20:00 (11/07) Irfaan (2)🏆3️⃣,873757,2018-05-15T20:52:52,Irfaan Moolla,Today,student,noncontent,conv
3,private_group,13:00 (10/07) Fatima (3)🏆3️⃣,1368973,2018-07-02T19:05:13,TZ Simphiwe Mfaba,Kulungile 💪,tutor,noncontent,conv
4,private_group,20:00 (09/07) Jenna (1)🏆4⃣,1234119,2018-06-18T20:12:31,ThishaBot,"Surprisingly, the patient seems satisfied with...",bot,content,non mcq


In [7]:
test.drop(columns=['text_type'], inplace=True)

In [8]:
X = train['text']
y = train['type']

In [9]:
print(X.shape)
print(y.shape)

(259,)
(259,)


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=32, test_size=0.2)

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)
print (x_train_vec.shape)
print (x_test_vec.shape)

(207, 2163)
(52, 2163)


## neural net

In [13]:
from keras import models
from keras import layers
from keras.layers import Dropout
from keras import optimizers
from keras.optimizers import SGD

Using TensorFlow backend.


In [14]:
# def create_model():
#     inputs = layers.Input(shape=(2163,))
#     hidden_1 = layers.Dense(units=16, activation='relu')(inputs)
#     dropout1 = layers.Dropout(0.2)(hidden_1)
#     hidden_2 = layers.Dense(units=16, activation='relu')(hidden_1)
#     dropout_2 = layers.Dropout(0.2)(hidden_2)
#     outputs = layers.Dense(1, activation='softmax')(hidden_2)

#     model= models.Model(inputs=inputs, outputs=outputs)
#     #model.summary()
#     model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')
#     return model

## parameter training

In [15]:
from keras import Sequential
from keras.layers import Dense
from keras.losses import binary_crossentropy

In [16]:
def create_model_2(hidden_layers=1,dropout_rate=0):
    model2 = Sequential()
    
    model2.add(Dense(16, input_shape=(2163,), activation='relu',))
    for i in range(hidden_layers):
        model2.add(Dense(16, activation='relu'))
        model2.add(Dropout(dropout_rate))
    model2.add(Dense(1, activation='sigmoid'))

    model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier

model2 = KerasClassifier(build_fn=create_model_2, verbose=0, epochs=32, batch_size=8, shuffle=True)

In [18]:
from sklearn.model_selection import GridSearchCV

hidden_layers= [2,4,6,8,10,12]
dropout_rate=[0.2,0.4,0.6,0.8]


param_grid2 = dict(hidden_layers=hidden_layers, dropout_rate=dropout_rate)

grid_2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1)

In [19]:
#grid2_pred = grid_2.fit(x_train_vec, y_train)

In [20]:
#print(grid2_pred.best_score_)
#print(grid2_pred.best_params_)

In [21]:
def create_model_3(neurons):
    model3 = Sequential()
    
    model3.add(Dense(neurons, input_shape=(2163,), activation='relu',))
    for i in range(4):
        model3.add(Dense(neurons, activation='relu'))
        model3.add(Dropout(0.2))
    model3.add(Dense(1, activation='sigmoid'))

    model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model3

In [22]:
model3 = KerasClassifier(build_fn=create_model_3, verbose=0, epochs=32, batch_size=8, shuffle=True)

In [23]:
neurons= [4,8,16,32,64]

param_grid3 = dict(neurons=neurons)

grid_3 = GridSearchCV(estimator=model3, param_grid=param_grid3, n_jobs=-1)

In [24]:
grid3_pred = grid_3.fit(x_train_vec, y_train)

In [25]:
print(grid3_pred.best_score_)
print(grid3_pred.best_params_)

0.8212560389352881
{'neurons': 64}


In [32]:
inputs = layers.Input(shape=(2163,))
hidden_1 = layers.Dense(units=2048, activation='relu')(inputs)
dropout1 = layers.Dropout(0.2)(hidden_1)
hidden_2 = layers.Dense(units=2048, activation='relu')(hidden_1)
dropout2 = layers.Dropout(0.2)(hidden_2)
hidden_3=layers.Dense(units=2048,activation='relu')(hidden_2)
dropout3=layers.Dropout(0.2)(hidden_3)
hidden_4=layers.Dense(units=2048,activation='relu')(hidden_3)
dropout4=layers.Dropout(0.2)(hidden_4)
outputs = layers.Dense(1, activation='sigmoid')(hidden_2)

network= models.Model(inputs=inputs, outputs=outputs)
network.summary()
network.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2163)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 2048)              4431872   
_________________________________________________________________
dense_23 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 2049      
Total params: 8,630,273
Trainable params: 8,630,273
Non-trainable params: 0
_________________________________________________________________


In [33]:
network.fit(x=x_train_vec, y=y_train, batch_size=8, epochs=32, verbose=0)
test_loss, test_acc = network.evaluate(x_test_vec, y_test)
print (test_loss)
print (test_acc)

Epoch 1/32
207/207 [==============================] - 3s 13ms/step - loss: 0.6205 - acc: 0.6957
Epoch 2/32
207/207 [==============================] - 1s 7ms/step - loss: 0.2139 - acc: 0.9372
Epoch 3/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1120 - acc: 0.9710
Epoch 4/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1380 - acc: 0.9710
Epoch 5/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1001 - acc: 0.9710
Epoch 6/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1017 - acc: 0.9710
Epoch 7/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1039 - acc: 0.9710
Epoch 8/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1135 - acc: 0.9710
Epoch 9/32
207/207 [==============================] - 1s 7ms/step - loss: 0.1003 - acc: 0.9710
Epoch 10/32
207/207 [==============================] - 1s 7ms/step - loss: 0.0990 - acc: 0.9710
Epoch 11/32
207/207 [===========================